In [ ]:
%matplotlib inline

In [ ]:
token = "b371402dc767cc83e41bc294b63f9586"
house = {
    "cellar": {
        "electricity": "fed676021dacaaf6a12a8dda7685be34"
    }
}

In [ ]:
import os
import tmpo

s = tmpo.Session(path=os.getcwd())
for room in house:
    for sensor in house[room]:
        s.add(house[room][sensor], token)

In [ ]:
s.debug = True

In [ ]:
s.sync()

In [ ]:
import pandas as pd
start = pd.Timestamp('20160101')

In [ ]:
df = s.dataframe([house[room]["electricity"]], head=start)

In [ ]:
import matplotlib.pyplot as plt

room = "cellar"
plt.figure(figsize=(16,10))
ax1=plt.subplot()
ax1.grid()
ax1.set_ylabel(u'Wh')
plt.title(room + " electricity")
df.plot(ax=ax1)